In [ ]:
df = pd.read_csv('Speed_Dating_Data.csv', encoding='latin1')
df = df.drop(['id', 'idg'], 1)
df = df.drop(['position', 'positin1', 'order', 'partner'], 1)
df = df.drop(['age_o', 'race_o', 'pf_o_att', 
              'pf_o_sin', 'pf_o_int',
              'pf_o_fun', 'pf_o_amb', 'pf_o_sha',
              'dec_o', 'attr_o', 'sinc_o', 'intel_o', 'fun_o',
              'amb_o', 'shar_o', 'like_o', 'prob_o','met_o'], 
             axis=1)
df = df.dropna(subset=['age'])
df.loc[:, 'field_cd'] = df.loc[:, 'field_cd'].fillna(19)
df = df.drop(['field'], axis=1)
field_cd = pd.get_dummies(df['field_cd'])
field_cd.columns = ['field_cd_{}'.format(i) for i in range(field_cd.shape[1])]
df = df.join(field_cd)
df = df.drop(['undergra'], axis=1)
df.loc[:, 'mn_sat'] = df.loc[:, 'mn_sat'].str.replace(',', '').astype(np.float)
df['mn_sat'] -= df['mn_sat'].mean()
df.loc[:, 'tuition'] = df.loc[:, 'tuition'].str.replace(',', '').astype(np.float)
df['tuition'] -= df['tuition'].mean()
df['race'] = df['race'].astype(np.int32)
race = pd.get_dummies(df['race'])
race.columns = ['race_{}'.format(i) for i in range(race.shape[1])]
df = df.join(race)
df = df.dropna(subset=['imprelig', 'imprace'])
df = df.drop(['from', 'zipcode'], axis=1)
df.loc[:, 'income'] = df.loc[:, 'income'].str.replace(',', '').astype(np.float)
df = df.drop(['income'], axis=1)
df = one_hot(df, 'goal')
df = df.drop(['date_3'], 1)
df = one_hot(df, 'date')
df = one_hot(df, 'go_out')
df.loc[:, 'career_c'] = df.loc[:, 'career_c'].fillna(18)
df = df.drop(['career'], axis=1)
df = one_hot(df, 'career_c')
df = df.drop(['sports','tvsports','exercise','dining','museums','art','hiking','gaming',
       'clubbing','reading','tv','theater','movies','concerts','music','shopping','yoga'], axis=1)
df = df.drop(['expnum'], axis=1)

feat = ['iid', 'wave', 'attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']
temp = df.drop_duplicates(subset=['iid', 'wave']).loc[:, feat]
temp.loc[:, 'totalsum'] = temp.iloc[:, 2:].sum(axis=1)
idx = ((temp.wave < 6) | (temp.wave > 9)) & (temp.totalsum < 99)
idx = ((temp.wave >= 6) & (temp.wave <= 9))
df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']].sum(axis=1)
df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']] = \
(df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']].T/df.loc[:, 'temp_totalsum'].T).T * 100

feat = ['iid', 'wave', 'attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']
temp = df.drop_duplicates(subset=['iid', 'wave']).loc[:, feat]
temp.loc[:, 'totalsum'] = temp.iloc[:, 2:].sum(axis=1)
idx = ((temp.wave < 6) | (temp.wave > 9)) & (temp.totalsum < 90) & (temp.totalsum != 0)
idx = ((temp.wave >= 6) & (temp.wave <= 9))
df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']].sum(axis=1)
df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']] = \
(df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']].T/df.loc[:, 'temp_totalsum'].T).T * 100
df = df.drop(['temp_totalsum'], axis=1)

for i in [4, 5]:
    feat = ['attr{}_1'.format(i), 'sinc{}_1'.format(i), 
            'intel{}_1'.format(i), 'fun{}_1'.format(i), 
            'amb{}_1'.format(i), 'shar{}_1'.format(i)]
    
    if i != 4:
        feat.remove('shar{}_1'.format(i))
    
    df = df.drop(feat, axis=1)
    
df['int_corr'] = df['int_corr'].fillna(df['int_corr'].mean())
df['field_cd'] = df['field_cd'].fillna(df['field_cd'].max())
df['mn_sat'] = df['mn_sat'].fillna(df['mn_sat'].mean())
df['tuition'] = df['tuition'].fillna(df['tuition'].mean())
df = df.dropna(subset=['pid'])
df = df.drop(['date'], 1)

In [ ]:
def one_hot(df, column):
    data = pd.get_dummies(df[column])
    data.columns = [column + '_{}'.format(i) for i in range(data.shape[1])]
    return df.join(data)

In [ ]:
columns = []
for column in df.columns:
    isnull = df[column].isnull().sum()
    if isnull:
        columns.append(column)
        print(column, df[column].isnull().sum())

In [ ]:
for column in columns:
    df[column] = df[column].fillna(df[column].mean())
    
df = df.drop(['wave'], axis=1)
df_male = df.query('gender == 1').drop_duplicates(subset=['iid', 'pid'])\
                                 .drop(['gender'], axis=1)\
                                 .dropna()
df_female = df.query('gender == 0').drop_duplicates(subset=['iid'])\
                                   .drop(['gender', 'match', 'int_corr', 'samerace'], axis=1)\
                                   .dropna()
        
df_female.columns = df_female.columns + '_f'
df = pd.merge(df_male, df_female, left_on=['pid'], right_on=['iid_f'], how='inner')